In [30]:
# Libraries
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from pre_process import pre_process_frame
from collections import Counter 
from imutils import paths
from pandas import DataFrame
import cv2
import glob
import cv2


In [2]:
# Import Model
vggface = VGGFace(model='senet50',include_top=False, input_shape=(224, 224, 3))
detector = cv2.CascadeClassifier('detector.xml')

In [55]:
# utils functions

def extract_face(filename,detector,required_size=(224, 224)):
    pixels = cv2.imread(filename)
    results = pre_process_frame(pixels,detector)
    if len(results) > 0:
        x1, y1, width, height = results
        x2, y2 = x1 + width, y1 + height
        face = pixels[y1:y2, x1:x2]
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = asarray(image)
        return face_array
    else:
        return []
def get_embeddings(filenames,detector):
    faces = []
    nomes = []
    for f in filenames:
        result = extract_face(f,detector)
        if len(result) > 0:
            nomes.append(f.split(os.path.sep)[-2])
            faces.append(result)
    samples = asarray(faces,'float32')
    samples = preprocess_input(samples, version=2)
    yhat = vggface.predict(samples)
    return yhat,nomes
def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0] 
# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
	# calculate distance between embeddings
	score = cosine(known_embedding, candidate_embedding)
	if score <= thresh:
		print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
	else:
		print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))
def get_name(known_embedding, candidate_embedding,names_collections, thresh=0.5):
    candidates = []
    for embedding,name in zip(known_embedding,names_collections):
        score = cosine(candidate_embedding, embedding)
        if score <= thresh:
            candidates.append(name)
    if len(candidates) > 0:
        name = most_frequent(candidates)
    else:
        name = 'Unknown'
    return name
def get_name_mean(known_embedding, candidate_embedding,names_collections,threshold = 0.5):
    distance = []
    candidate = []
    for embedding,name in zip(known_embedding,names_collections):
        score = cosine(candidate_embedding, embedding)
        distance.append(score)
        candidate.append(name)
    dic = {
            'Distance':distance,
            'candidate':candidate
    }
    df = DataFrame(dic)
    df_mean = df.groupby(['candidate'])['Distance'].agg(['mean']).reset_index()
    minimo = df_mean['mean'].min()
    row_min = df_mean[df_mean['mean'] == minimo]
    min_dist = row_min.mean().to_list()[0]
    name = row_min.candidate.to_list()[0]
    # give name
    if min_dist < threshold:
        return name
    else:
        return 'Unknown'    

In [6]:
# Testing with everyone
database,names = get_embeddings(list(paths.list_images("Test/")),detector)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [10]:
# test personal
entrance,names_entrance = get_embeddings(list(paths.list_images("Train/")),detector)

In [22]:
# for election
data = {
    'name':[],
    'predict':[],
    'threshold':[]
}
for threshold in range(10,70):
    for embedding,nome in zip(entrance,names_entrance):
        predict = get_name(database,embedding,names,(threshold/100))
        data['name'].append(nome)
        data['predict'].append(predict)
        data['threshold'].append((threshold/100))


In [62]:
# for mean
data_mean = {
    'name':[],
    'predict':[],
    'threshold':[]
}
for threshold in range(10,70):
    for embedding,nome in zip(entrance,names_entrance):
        predict = get_name_mean(database,embedding,names,(threshold/100))
        data_mean['name'].append(nome)
        data_mean['predict'].append(predict)
        data_mean['threshold'].append((threshold/100))